<a href="https://colab.research.google.com/github/gulabpatel/FastAPI/blob/main/00%3A%20FastAPI_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code walkthrough :

1. https://www.youtube.com/watch?v=WU65u9d-97c
2. https://www.youtube.com/watch?v=b5F667g1yCk&t=18s

In [ ]:
!pip install uvicorn fastapi colabcode

In [2]:
%%writefile BankNotes.py
from pydantic import BaseModel
# 2. Class which describes Bank Notes measurements
class BankNote(BaseModel):
    variance: float 
    skewness: float 
    curtosis: float 
    entropy: float

Writing BankNotes.py


In [3]:
from colabcode import ColabCode
from fastapi import FastAPI
cc = ColabCode(port=12000, code=False)

Build Model

In [4]:
import pandas as pd
import numpy as np
df=pd.read_csv('https://raw.githubusercontent.com/gulabpatel/FastAPI/master/dataset/BankNote_Authentication.txt')

### Independent and Dependent features
X=df.iloc[:,:-1]
y=df.iloc[:,-1]

### Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

### Implement Random Forest classifier
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()
classifier.fit(X_train,y_train)

## Prediction
y_pred=classifier.predict(X_test)

### Check Accuracy
from sklearn.metrics import accuracy_score
score=accuracy_score(y_test,y_pred)

### Create a Pickle file using serialization 
import pickle
pickle_out = open("classifier.pkl","wb")
pickle.dump(classifier, pickle_out)

In [6]:
import uvicorn
from BankNotes import BankNote
import numpy as np
import pickle
import pandas as pd
# 2. Create the app object
app = FastAPI()
pickle_in = open("classifier.pkl","rb")
classifier=pickle.load(pickle_in)

# 3. Index route
@app.get('/')
def index():
    return {'message': 'Hello, World'}


@app.get('/{name}')
def get_name(name: str):
    return {'Welcome! To Future of AI': f'{name}'}

# 3. Expose the prediction functionality, make a prediction from the passed
#    JSON data and return the predicted Bank Note with the confidence
@app.post('/predict')
def predict_banknote(data:BankNote):
    data = data.dict()
    variance=data['variance']
    skewness=data['skewness']
    curtosis=data['curtosis']
    entropy=data['entropy']
   # print(classifier.predict([[variance,skewness,curtosis,entropy]]))
    prediction = classifier.predict([[variance,skewness,curtosis,entropy]])
    if(prediction[0]>0.5):
        prediction="Fake note"
    else:
        prediction="Its a Bank note"
    return {
        'prediction': prediction
    }


In [ ]:
cc.run_app(app=app) 

INFO:     Started server process [768]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://bc28baaf9b10.ngrok.io" -> "http://localhost:12000"
